## Deliverable 2. Create a Customer Travel Destinations Map.

In [75]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [76]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,83.89,84,20,0.00,few clouds
1,1,Mataura,NZ,-46.1927,168.8643,49.53,99,100,2.10,moderate rain
2,2,Bethel,US,41.3712,-73.4140,68.36,92,90,1.01,overcast clouds
3,3,Albany,US,42.6001,-73.9662,56.23,95,100,2.64,overcast clouds
4,4,Kiruna,SE,67.8557,20.2251,22.12,93,90,1.14,overcast clouds


In [77]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [78]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Plouzane,FR,48.3833,-4.6167,62.92,74,75,12.66,broken clouds
18,18,Castro,BR,-24.7911,-50.0119,63.99,74,94,6.38,overcast clouds
57,57,Oneonta,US,42.4529,-75.0638,62.11,90,95,7.00,overcast clouds
77,77,Bereda,ES,43.2686,-7.5406,63.14,76,60,3.38,broken clouds
97,97,Lima,PE,-12.0432,-77.0282,61.90,85,90,8.05,overcast clouds
110,110,Mapimi,MX,25.8167,-103.8500,61.57,36,1,2.04,clear sky
139,139,Guerrero Negro,MX,27.9769,-114.0611,62.49,83,4,5.86,clear sky
141,141,Taltal,CL,-25.4000,-70.4833,61.23,75,24,4.00,few clouds
143,143,Airai,TL,-8.9266,125.4092,63.36,89,99,0.13,overcast clouds
150,150,Rawson,AR,-43.3002,-65.1023,61.25,25,9,29.71,clear sky


In [80]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             36
City                   36
Country                36
Lat                    36
Lng                    36
Max Temp               36
Humidity               36
Cloudiness             36
Wind Speed             36
Current Description    36
dtype: int64

In [81]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [83]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Plouzane,FR,62.92,broken clouds,48.3833,-4.6167,
18,Castro,BR,63.99,overcast clouds,-24.7911,-50.0119,
57,Oneonta,US,62.11,overcast clouds,42.4529,-75.0638,
77,Bereda,ES,63.14,broken clouds,43.2686,-7.5406,
97,Lima,PE,61.90,overcast clouds,-12.0432,-77.0282,
110,Mapimi,MX,61.57,clear sky,25.8167,-103.8500,
139,Guerrero Negro,MX,62.49,clear sky,27.9769,-114.0611,
141,Taltal,CL,61.23,few clouds,-25.4000,-70.4833,
143,Airai,TL,63.36,overcast clouds,-8.9266,125.4092,
150,Rawson,AR,61.25,clear sky,-43.3002,-65.1023,


In [84]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.


In [85]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)


In [86]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [87]:
clean_hotel_df = hotel_df

In [88]:
# 8a. Create the output File (CSV)
clean_hotel_df.to_csv('WeatherPy_vacation.csv')


In [90]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Desciption</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [91]:
# 11a. Add a marker layer for each city to the map. 
#locations = hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))